In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [ ]:
covid_df = pd.read_csv("covid.csv")
covid_df = covid_df.drop(len(covid_df) - 1, axis=0)
covid_df

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2.0,1.0,04-05-2020,02-05-2020,9999-99-99,97.0,2.0,27.0,97.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,97.0
1,1009bf,2.0,1.0,19-03-2020,17-03-2020,9999-99-99,97.0,2.0,24.0,97.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,99.0,1.0,97.0
2,167386,1.0,2.0,06-04-2020,01-04-2020,9999-99-99,2.0,2.0,54.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,99.0,1.0,2.0
3,0b5948,2.0,2.0,17-04-2020,10-04-2020,9999-99-99,2.0,1.0,30.0,97.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,99.0,1.0,2.0
4,0d01b5,1.0,2.0,13-04-2020,13-04-2020,22-04-2020,2.0,2.0,60.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,99.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394600,1b36e2,1.0,1.0,26-06-2020,26-06-2020,9999-99-99,97.0,2.0,31.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,99.0,2.0,97.0
394601,01a289,2.0,1.0,12-06-2020,11-06-2020,9999-99-99,97.0,2.0,29.0,97.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,97.0
394602,0a874d,1.0,1.0,11-04-2020,08-04-2020,9999-99-99,97.0,2.0,63.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,97.0
394603,020dc0,2.0,1.0,20-06-2020,17-06-2020,9999-99-99,97.0,2.0,20.0,97.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,99.0,2.0,97.0


In [ ]:
covid_df['date_died'] = (covid_df['date_died'] != "9999-99-99") * 1
covid_df = covid_df.stack()
covid_df[covid_df == 97] = 3
covid_df[covid_df == 98] = 3
covid_df[covid_df == 99] = 3
covid_df = covid_df.unstack()


In [ ]:
covid_df.head(6)

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,0,3,2,27,3,2,2,2,2,2,2,2,2,2,2,2,1,3
1,1009bf,2,1,19-03-2020,17-03-2020,0,3,2,24,3,2,2,2,2,2,2,2,2,2,2,3,1,3
2,167386,1,2,06-04-2020,01-04-2020,0,2,2,54,2,2,2,2,2,2,2,2,1,2,2,3,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,0,2,1,30,3,2,2,2,2,2,2,2,2,2,2,3,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,1,2,2,60,2,1,2,2,2,1,2,1,2,2,2,3,1,2
5,1beec8,2,2,16-04-2020,16-04-2020,1,2,1,47,3,1,2,2,2,2,2,2,2,2,2,3,1,1


In [ ]:
covid_df = covid_df.drop(['id', 'entry_date', 'date_symptoms'], axis=1)
covid_df.rename(columns={'date_died':'is_dead'}, inplace=True)


In [ ]:
covid_df

,sex,patient_type,is_dead,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,2,1,0,3,2,27,3,2,2,2,2,2,2,2,2,2,2,2,1,3
1,2,1,0,3,2,24,3,2,2,2,2,2,2,2,2,2,2,3,1,3
2,1,2,0,2,2,54,2,2,2,2,2,2,2,2,1,2,2,3,1,2
3,2,2,0,2,1,30,3,2,2,2,2,2,2,2,2,2,2,3,1,2
4,1,2,1,2,2,60,2,1,2,2,2,1,2,1,2,2,2,3,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394600,1,1,0,3,2,31,2,2,2,2,2,2,2,2,2,2,2,3,2,3
394601,2,1,0,3,2,29,3,2,2,1,2,2,2,2,2,2,2,1,2,3
394602,1,1,0,3,2,63,2,2,2,2,2,1,2,2,1,2,2,2,2,3
394603,2,1,0,3,2,20,3,2,2,2,2,2,2,2,2,2,2,3,2,3


In [ ]:
X = covid_df.drop('is_dead', axis=1).to_numpy()
y = covid_df['is_dead'].to_numpy()

In [ ]:
from numpy.core.arrayprint import array_str
print('X:' + array_str(X))
print('y:' + array_str(y))

X:[[2.0 1.0 3 ... 2.0 1.0 3]
 [2.0 1.0 3 ... 3 1.0 3]
 [1.0 2.0 2.0 ... 3 1.0 2.0]
 ...
 [1.0 1.0 3 ... 2.0 2.0 3]
 [2.0 1.0 3 ... 3 2.0 3]
 [2.0 1.0 3 ... 3 2.0 3]]
y:[0 0 0 ... 0 0 0]


In [ ]:
X = X.astype(int)
y = np.array([y.astype(int)]).T

In [ ]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=0.33)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers

In [ ]:
K.clear_session()

In [ ]:
model = Sequential()

nh = 100
ntr = Xtr.shape[0]
n_feat = Xtr.shape[1]

model.add(Dense(units=nh, input_shape=(n_feat,), activation='sigmoid', name='input'))
model.add(Dense(units=nh, input_shape=(nh,), activation='sigmoid', name='hidden'))
model.add(Dense(units=2, input_shape=(nh,), activation='softmax', name='output'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 100)               2000      
                                                                 
 hidden (Dense)              (None, 100)               10100     
                                                                 
 output (Dense)              (None, 2)                 202       
                                                                 
Total params: 12,302
Trainable params: 12,302
Non-trainable params: 0
_________________________________________________________________


In [ ]:
opt = optimizers.Adam(lr=0.001)
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
hist = model.fit(Xtr, ytr, epochs=100, batch_size=1000, shuffle=True, validation_data=(Xts, yts))

Epoch 1/100
265/265 [==============================] - 3s 4ms/step - loss: 0.2143 - accuracy: 0.9221 - val_loss: 0.1599 - val_accuracy: 0.9276
Epoch 2/100
265/265 [==============================] - 1s 3ms/step - loss: 0.1562 - accuracy: 0.9286 - val_loss: 0.1541 - val_accuracy: 0.9295
Epoch 3/100
265/265 [==============================] - 1s 3ms/step - loss: 0.1537 - accuracy: 0.9295 - val_loss: 0.1524 - val_accuracy: 0.9299
Epoch 4/100
265/265 [==============================] - 1s 3ms/step - loss: 0.1531 - accuracy: 0.9297 - val_loss: 0.1539 - val_accuracy: 0.9271
Epoch 5/100
265/265 [==============================] - 1s 3ms/step - loss: 0.1528 - accuracy: 0.9296 - val_loss: 0.1519 - val_accuracy: 0.9307
Epoch 6/100
265/265 [==============================] - 1s 3ms/step - loss: 0.1526 - accuracy: 0.9300 - val_loss: 0.1506 - val_accuracy: 0.9308
Epoch 7/100
265/265 [==============================] - 1s 3ms/step - loss: 0.1520 - accuracy: 0.9301 - val_loss: 0.1501 - val_accuracy: 0.9312

In [ ]:
yhat_prob = model.predict(Xts)

In [ ]:
yhat_prob

array([[9.9961460e-01, 3.8540064e-04],
       [9.9987340e-01, 1.2660913e-04],
       [9.9829847e-01, 1.7015217e-03],
       ...,
       [9.4274932e-01, 5.7250667e-02],
       [8.6719018e-01, 1.3280986e-01],
       [9.4420540e-01, 5.5794626e-02]], dtype=float32)

In [ ]:
yhat_class = (yhat_prob < 0.5) * 0 + (yhat_prob > 0.5) * 1

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
yts_class = enc.fit(yts)

In [ ]:
yhat = enc.inverse_transform(yhat_class)
print(yhat)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:

nh = 100
ntr = Xtr.shape[0]
n_feat = Xtr.shape[1]
acc = np.zeros((10, 5))
lrs = np.logspace(-5, -1, num=5, endpoint=True)

for i in range(10):
  K.clear_session()
  model = Sequential()

  model.add(Dense(units=nh, input_shape=(n_feat,), activation='sigmoid', name='input'))
  
  for j in range(i):
    model.add(Dense(units=nh, input_shape=(nh,), activation='sigmoid', name='hidden' + str(j)))
  model.add(Dense(units=2, input_shape=(nh,), activation='softmax', name='output'))

  for id, k in enumerate(lrs):
    opt = optimizers.Adam(lr=k)
    model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

    hist = model.fit(Xtr, ytr, epochs=100, batch_size=1000, shuffle=True, validation_data=(Xts, yts))
    acc[i,id] = np.max(hist.history['val_accuracy'])

np.argmax(acc, axis=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
380/380 [==============================] - 2s 4ms/step - loss: 1.3365 - accuracy: 0.0640 - val_loss: 0.9791 - val_accuracy: 0.0635
Epoch 2/100
380/380 [==============================] - 1s 4ms/step - loss: 0.7418 - accuracy: 0.4238 - val_loss: 0.5554 - val_accuracy: 0.9365
Epoch 3/100
380/380 [==============================] - 1s 4ms/step - loss: 0.4496 - accuracy: 0.9360 - val_loss: 0.3693 - val_accuracy: 0.9365
Epoch 4/100
380/380 [==============================] - 1s 4ms/step - loss: 0.3260 - accuracy: 0.9360 - val_loss: 0.2923 - val_accuracy: 0.9365
Epoch 5/100
380/380 [==============================] - 1s 4ms/step - loss: 0.2746 - accuracy: 0.9360 - val_loss: 0.2598 - val_accuracy: 0.9365
Epoch 6/100
380/380 [==============================] - 1s 4ms/step - loss: 0.2528 - accuracy: 0.9360 - val_loss: 0.2459 - val_accuracy: 0.9365
Epoch 7/100
380/380 [==============================] - 1s 4ms/step - loss: 0.2436 - accuracy: 0.9360 - 

array([2, 3, 3, 3, 3, 3, 2, 3, 2, 2])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 100)               1100      
                                                                 
 hidden2 (Dense)             (None, 100)               10100     
                                                                 
 hidden3 (Dense)             (None, 100)               10100     
                                                                 
 hidden4 (Dense)             (None, 100)               10100     
                                                                 
 output (Dense)              (None, 2)                 202       
                                                                 
Total params: 31,602
Trainable params: 31,602
Non-trainable params: 0
_________________________________________________________________


In [ ]:
acc

array([[0.94397765, 0.94512218, 0.94572651, 0.94561422, 0.94505262],
       [0.94446969, 0.94530934, 0.94563025, 0.94575328, 0.93645811],
       [0.94391882, 0.94532007, 0.94567305, 0.94597787, 0.93645811],
       [0.94345891, 0.94539493, 0.94569445, 0.94579607, 0.93645811],
       [0.94381189, 0.94523984, 0.94563025, 0.94579071, 0.93645811],
       [0.94314337, 0.94515961, 0.94568372, 0.94574791, 0.93645811],
       [0.94278502, 0.94525051, 0.94565701, 0.94560349, 0.93645811],
       [0.94281709, 0.94525051, 0.94567841, 0.9458549 , 0.93645811],
       [0.9432236 , 0.94518101, 0.94562489, 0.94529331, 0.93645811],
       [0.93645811, 0.94514358, 0.94560349, 0.94513285, 0.93645811]])